In [24]:
import json

In [25]:
from pyspark.sql import SparkSession

In [26]:
config = json.load(open("config.json"))

In [27]:
spark = SparkSession.builder\
    .config("spark.jars", config['JDBC_PATH']) \
    .config("spark.executor.extraClassPath", config['JDBC_PATH'])\
    .getOrCreate()

In [28]:
dbtable = "store"

In [29]:
df_store = spark.read.format("jdbc")\
        .option("url",config['SRC_DB_URL'])\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("dbtable", dbtable)\
        .option("user", config['SRC_DB_USER'])\
        .option("password", config['SRC_DB_PASSWORD'])\
        .load()

In [30]:
df_store.createOrReplaceTempView(dbtable)

In [31]:
dbtable = "staff"

In [32]:
df_staff = spark.read.format("jdbc")\
        .option("url",config['SRC_DB_URL'])\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("dbtable", dbtable)\
        .option("user", config['SRC_DB_USER'])\
        .option("password", config['SRC_DB_PASSWORD'])\
        .load()

In [33]:
df_staff.createOrReplaceTempView(dbtable)

In [34]:
dbtable = "address"

In [35]:
df_address = spark.read.format("jdbc")\
        .option("url",config['SRC_DB_URL'])\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("dbtable", dbtable)\
        .option("user", config['SRC_DB_USER'])\
        .option("password", config['SRC_DB_PASSWORD'])\
        .load()

In [36]:
df_address.createOrReplaceTempView(dbtable)

In [37]:
dbtable = "city"

In [38]:
df_city = spark.read.format("jdbc")\
        .option("url",config['SRC_DB_URL'])\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("dbtable", dbtable)\
        .option("user", config['SRC_DB_USER'])\
        .option("password", config['SRC_DB_PASSWORD'])\
        .load()

In [39]:
df_city.createOrReplaceTempView(dbtable)

In [40]:
dbtable = "country"

In [41]:
df_country = spark.read.format("jdbc")\
        .option("url",config['SRC_DB_URL'])\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("dbtable", dbtable)\
        .option("user", config['SRC_DB_USER'])\
        .option("password", config['SRC_DB_PASSWORD'])\
        .load()

In [42]:
df_country.createOrReplaceTempView(dbtable)

In [43]:
dim_staff = spark.sql("""
SELECT
ROW_NUMBER() OVER(ORDER BY stf.staff_id) AS staff_id,
stf.staff_id AS staff_key,
CONCAT(stf.first_name, " ", stf.last_name) AS staff_name,
CONCAT(stf_mngr.first_name, " ", stf_mngr.last_name) AS staff_manager_name,
a.address AS staff_address,
a.postal_code AS staff_postal_code,
ci.city AS staff_city,
co.country AS staff_country
FROM staff AS stf
JOIN staff AS stf_mngr ON stf.manager_id = stf_mngr.staff_id
JOIN store AS s ON stf.store_id = s.store_id
JOIN address AS a ON s.address_id = a.address_id
JOIN city AS ci ON a.city_id = ci.city_id
JOIN country co ON ci.country_id = co.country_id 
""")

In [44]:
dim_staff.show()

24/01/09 00:58:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:58:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:58:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:58:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:58:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:58:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 0

+--------+---------+--------------------+--------------------+--------------------+-----------------+------------+--------------+
|staff_id|staff_key|          staff_name|  staff_manager_name|       staff_address|staff_postal_code|  staff_city| staff_country|
+--------+---------+--------------------+--------------------+--------------------+-----------------+------------+--------------+
|       1|        1|   Giovanna Barkholz|   Giovanna Barkholz|         Wolności 90|           42-625|    Katowice|        Poland|
|       2|        2|       Viktor Soukup|       Viktor Soukup|         Aviatická 6|           161 00|      Prague|Czech Republic|
|       3|        3|   Christine Kreusel|   Christine Kreusel|          Mošnov 401|           742 51|     Ostrawa|Czech Republic|
|       4|        4|Cornelia Holzapfe...|Cornelia Holzapfe...|  Flughafenstraße 32|            70629|   Stuttgart|       Germany|
|       5|        5|         Miłosz Berg|         Miłosz Berg|   Flughafenstr. 1-3|       

In [45]:
dim_staff.write.format("jdbc")\
        .option("url",config['DST_DB_URL'])\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("dbtable", "dim_staff")\
        .option("user", config['DST_DB_USER'])\
        .option("password", config['DST_DB_PASSWORD'])\
        .save()

24/01/09 00:58:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:58:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:58:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:59:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:59:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:59:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 0